In [1]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

In [2]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_rna


In [3]:
import os
json_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"hai-gcp-genomic-beb9207a40b8.json")
!ls $json_path

/home/eila/git_igvf/atomic-workflows/nextflow/hai-gcp-genomic-beb9207a40b8.json


In [4]:
# # in the command line. same env
import os
json_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"hai-gcp-genomic-beb9207a40b8.json")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_path
!gcloud auth activate-service-account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS

Activated service account credentials for: [eila-batch@hai-gcp-genomic.iam.gserviceaccount.com]


In [5]:
!gcloud auth list
project_name = !gcloud config get-value project
project_name = project_name[0]
project_name
# gcloud config set account


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com
        eila@stanford.edu

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



'hai-gcp-genomic'

In [6]:
import os
os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["TOWER_WORKSPACE_ID"]='247507587970818'


In [7]:
!echo $TOWER_ACCESS_TOKEN
!echo $TOWER_WORKSPACE_ID

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz
247507587970818


In [8]:
# !chmod +x ./bin/scrna_qc_metrics.py
import os
bin_fld = os.path.join(os.getcwd(),"bin")
bin_fld

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_rna/bin'

In [9]:
# !chmod +x {os.path.join(bin_fld, "scrna_qc_metrics.py")}
# !chmod +x {os.path.join(bin_fld, "scrna_plot_qc_metrics.R")}
# !chmod +x {os.path.join(bin_fld, "barcode_rank_functions.R")}
!chmod +x {os.path.join(bin_fld, "cellatlas_build.sh")}
!chmod +x {os.path.join(bin_fld, "cellatlas_create_jq_commands_txt.sh")}
!chmod +x {os.path.join(bin_fld, "rna_kb_ref_with_jq_commands.sh")}
!chmod +x {os.path.join(bin_fld, "seqspec_retrieve_technology.sh")}
!chmod +x {os.path.join(bin_fld, "rna_kb_count_execution.sh")}
!chmod +x {os.path.join(bin_fld, "seqspec_run_modify.sh")}
!chmod +x {os.path.join(bin_fld, "rna_modify_kb_count_output_barcode_h5.py")}
!chmod +x {os.path.join(bin_fld, "rna_modify_kb_count_cell_gene_barcode.sh")}
!chmod +x {os.path.join(bin_fld, "cellatlas_jq_run_info_and_inspect.sh")}


In [10]:
!nextflow info

  Version: 23.10.1 build 5891
  Created: 12-01-2024 22:01 UTC 
  System: Linux 5.10.0-27-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.1-internal+0-adhoc..src
  Encoding: UTF-8 (UTF-8)



In [11]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Navigate two folders above
two_folders_above = os.path.abspath(os.path.join(current_directory, '..', '..'))

nf_data_base_path = os.path.join(two_folders_above,'nf_data')
# Print the result
print(nf_data_base_path)


/home/eila/git_igvf/atomic-workflows/nextflow/nf_data


In [12]:
# nf_data_project="test_atac_pipeline"
# nf_data_project="test_rna_pipeline"
nf_data_project="wdl_rna_pipeline"

In [13]:
# VM
os.path.exists(os.path.join(nf_data_base_path,nf_data_project,"vm_atac_fastqs_files.tsv"))
os.path.join(nf_data_base_path,nf_data_project,"vm_rna_input_files.tsv")
# /home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_rna_pipeline/

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_rna_pipeline/vm_rna_input_files.tsv'

In [14]:
# !ls /home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_rna_pipeline/vm_rna_input_files.tsv

In [15]:
!cp ../../config_files/docker-google-batch.config ../workflow_fg_single_cell_rna/conf/


In [16]:
# Write the configuration file
with open("nextflow_rna.config", "w") as config_file:
    config_file.write(f"""
manifest {{
  description = 'A nextflow pipeline for fg single cell' 
  mainScript = 'nf_wf_fg_singlecell_rna.nf'
}}

profiles {{
  google_batch_profile {{
    workDir = 'gs://eila-nextflow-bucket/work/RNA'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    executor = 'google-batch'
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    google.project = '{project_name}'
    google.location = 'us-central1'
    google.batch.spot = true
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }}
}}

// Define ENV_SYNAPSE_TOKEN as a parameter
params.ORGANISM = 'human'
params.FASTQS_SPEC_CH = '{nf_data_base_path}/{nf_data_project}/vm_rna_input_files.tsv'
params.GENOME_FASTA_GZ = '{nf_data_base_path}/{nf_data_project}/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz'
params.GENES_GTF = '{nf_data_base_path}/{nf_data_project}/genes.gtf'
params.CELLATLAS_BUILD_SCRIPT = 'cellatlas_build.sh'
params.JQ_CELLATLAS_BUILD_JSON_SCRIPT = 'cellatlas_create_jq_commands_txt.sh'
params.KB_REF_WITH_JQ_COMMANDS='rna_kb_ref_with_jq_commands.sh'
params.TRANSCRIPTOME_FA='{nf_data_base_path}/{nf_data_project}/transcriptome.fa'

params.EXECUTE_SEQSPEC_MODIFY='seqspec_run_modify.sh'
params.RETRIEVE_SEQSPEC_TECHNOLOGY='seqspec_retrieve_technology.sh'
params.EXECUTE_KB_COUNT_RNA_SCRIPT='rna_kb_count_execution.sh'
params.KB_COUNT_CPUS=16
//Debug until I have a complete seqspec
params.KB_COUNT_TECHNOLGY_FILE_DEBUG='{nf_data_base_path}/{nf_data_project}/technology_file.txt'

// STEP 7

params.RNA_MODIFY_KB_COUNT_H5AD_WITH_SUBPOOL='rna_modify_kb_count_output_barcode_h5.py'
params.RNA_MODIFY_CELL_GENE_WITH_SUBPOOL='rna_modify_kb_count_cell_gene_barcode.sh'
// DEBUG
params.adata_out_h5ad_debug='{nf_data_base_path}/{nf_data_project}/out/counts_unfiltered/adata.h5ad'
params.kb_count_run_info_json_debug='{nf_data_base_path}/{nf_data_project}/out/run_info.json'
params.kb_count_inspect_json_debug='{nf_data_base_path}/{nf_data_project}/out/inspect.json'
params.cells_x_genes_barcodes_out_debug='{nf_data_base_path}/{nf_data_project}/out/counts_unfiltered/cells_x_genes.barcodes.txt'
// STEP 8
params.RNA_JQ_CELLATLAS_OUTPUT_TO_LOG='cellatlas_jq_run_info_and_inspect.sh'
params.LOG_FILE_NAME='BMMC-single-donor.rna.log.hg38.txt'
// params.GENOME_FASTA = 'http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
// params.GENOME_GZ_GTF = 'http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
// params.ENV_SYNAPSE_TOKEN = "nextflow secrets get SYNAPSE".execute().text.trim()
// params.SYNAPSE_CONFIG = '{nf_data_base_path}/{nf_data_project}nf_wf_fg_singlecell/synapse.config'
// params.SYNAPSE_IDS = '{nf_data_base_path}/{nf_data_project}/nf_wf_fg_singlecell/synapse_ids.csv'
// params.SUBPOOL = 'no_subpool'
// params.UMI_CUTOFF = 2
// params.GENE_CUTOFF = 3
// params.SC_RNA_QC_SCRIPT = 'scrna_qc_metrics.py'
// params.SC_RNA_QC_PLOT_SCRIPT = 'scrna_plot_qc_metrics.R'
// params.SC_RNA_QC_PLOT_HELPER_SCRIPT = 'barcode_rank_functions.R'
// params.RNA_DEBUG_QC = '{nf_data_base_path}/{nf_data_project}/nf_workflow/workflow_fg_single_cell_rna/sc_rna_qc_calculate.tsv'
""")


In [17]:
# !rm -r /home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_rna_pipeline/out

In [18]:
# !rm -r /home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_rna_pipeline/out/

In [19]:
# !mv /home/eila/git_igvf/atomic-workflows/nextflow/nf_data/test_rna_pipeline/out /home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_rna_pipeline/

In [20]:
# !ls /home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_rna_pipeline
# file:///home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_rna_pipeline/out/counts_unfiltered/adata.h5ad 

In [21]:
# !rm -rf /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna/work
# !gsutil -m rm -r gs://eila-nextflow-bucket/work/RNA


In [22]:
!nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_rna.config -profile google_batch_profile -with-tower -resume
# -with-dag

# -resume 
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local -resume



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile  -with-tower

# docker_local



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile




# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile docker_profile

N E X T F L O W  ~  version 23.10.1
Launching `nf_wf_fg_singlecell_rna.nf` [focused_borg] DSL2 - revision: 542c54ee0a
in the RNA workflow
[-        ] process > run_cellatlas_build         -
[-        ] process > run_jq_on_cellatlas         -
[-        ] process > run_kb_ref_with_jq_commands -
[-        ] process > run_seqspec_modify_rna      -
after run_seqspec_modify_rna
Calling run_add_subpool_to_rna_kb_cout_outputs with the following inputs:
params.RNA_MODIFY_KB_COUNT_H5AD_WITH_SUBPOOL: rna_modify_kb_count_output_barcode_h5.py
params.RNA_MODIFY_CELL_GENE_WITH_SUBPOOL: rna_modify_kb_count_cell_gene_barcode.sh
adata_out_h5ad: DataflowVariable(value=null)
cells_x_genes_barcodes_out: DataflowVariable(value=null)

[-        ] process > run_cellatlas_build            -
[-        ] process > run_jq_on_cellatlas            -
[-        ] process > run_kb_ref_with_jq_commands    -
[-        ] process > run_seqspec_modify_rna         -
[-        ] process > run_add_subpool_to_rna_kb_c... -
aft

In [23]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_rna


In [24]:
!ls ../../nf_processes/nf_prcs_rna_kb_count_subpool_update.nf

../../nf_processes/nf_prcs_rna_kb_count_subpool_update.nf


In [25]:
!cat .nextflow.log 

Mar-07 17:09:33.646 [main] DEBUG nextflow.cli.Launcher - $> nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_rna.config -profile google_batch_profile -with-tower -resume
Mar-07 17:09:33.929 [main] INFO  nextflow.cli.CmdRun - N E X T F L O W  ~  version 23.10.1
Mar-07 17:09:33.967 [main] DEBUG nextflow.plugin.PluginsFacade - Setting up plugin manager > mode=prod; embedded=false; plugins-dir=/home/eila/.nextflow/plugins; core-plugins: nf-amazon@2.1.4,nf-azure@1.3.3,nf-cloudcache@0.3.0,nf-codecommit@0.1.5,nf-console@1.0.6,nf-ga4gh@1.1.0,nf-google@1.8.3,nf-tower@1.6.3,nf-wave@1.0.1
Mar-07 17:09:33.985 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Enabled plugins: []
Mar-07 17:09:33.988 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Disabled plugins: []
Mar-07 17:09:33.994 [main] INFO  org.pf4j.DefaultPluginManager - PF4J version 3.4.1 in 'deployment' mode
Mar-07 17:09:34.015 [main] INFO  org.pf4j.AbstractPluginManager - No plugins
Mar-07 17:09:34.049 [main] DEBUG nextflow.config.

In [26]:
import datetime
now = datetime.datetime.now()
print(now)

2024-03-07 17:12:33.650271


In [27]:

# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume 
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume 886dc4ef-ff60-46d0-aecb-8f1ab56296a3

# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local -resume





# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume




# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile google_batch_profile -resume

In [28]:
# !nextflow log
# use the session id with the resume

In [29]:
# !cat .nextflow.log 